### Run BASiCS for estimation method comparisons

In [5]:
library(Seurat)
library(SeuratData)
library(cowplot)
library(patchwork)
library(scHOT)
library(tidyverse)
library(dplyr)
library(BiocParallel)
library(readr)
library(Matrix)

Attaching SeuratObject

Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element 

In [6]:
library(Seurat)
library(SeuratData)
library(SeuratDisk)

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [7]:
library(BASiCS)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQR

### Convert h5ad to seurat objects for simulations

In [ ]:
setwd('/data_volume/memento/simulation/estimation/')

downsampling_rates = c(0.01, 0.025, 0.05, 0.1, 0.15, 0.2, 0.25, 0.35, 0.5)
# downsampling_rates = c(0.05, 0.1, 0.15, 0.2, 0.25, 0.35, 0.5)

num_replicates = 1
# downsampling_rates = c(1.0, 0.3)

for (downsample_rate in downsampling_rates){
    
    for (replicate in seq(0,num_replicates-1)){
        filename = paste(
            'estimation_',
            format(downsample_rate, nsmall = 1),
            '.h5ad',
            sep=''
        )
        Convert(filename, dest = "h5seurat", overwrite = TRUE, verbose = FALSE)
        
    }
}

### BASiCS for simulations

In [6]:
runBASiCS <- function(seurat, stat){
    Idents(seurat) <- 'stim_indicator'
    dat <- seurat
    dat$batch <- 1
    dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)),
    colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 1000, Thin = 10, Burn = 500, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = FALSE, Regression = TRUE,Threads = getOption("Ncpus", 10),)
    df = as.data.frame(displayChainBASiCS(Chain, Param = stat))
    return(df)
}

In [ ]:
for (downsample_rate in downsampling_rates){
    print(downsample_rate)
    
    for (replicate in seq(0,num_replicates-1)){
        infilename = paste(
            'estimation_',
            format(downsample_rate, nsmall = 1),
            '.h5seurat',
            sep=''
        )
        prefix <- 'BASiCS_estimation_'
        suffix <- paste(
            '_',
            format(downsample_rate, nsmall = 1),
            '.csv',
            sep=''
        )

        data_obj <- LoadH5Seurat(infilename, verbose = FALSE)
        
        mu <- runBASiCS(data_obj, 'mu')
        write_csv(mu, file = paste(prefix, 'mu', suffix, sep=''))
        
        delta <- runBASiCS(data_obj, 'delta')
        write_csv(delta, file = paste(prefix, 'delta', suffix, sep=''))

    }
}

### run BASiCS for smFISH comparison

In [3]:
setwd('/data_volume/memento/saver/')
data_path <- '/data_volume/memento/saver/'

In [10]:
Convert('dropseq.h5ad', dest = "h5seurat", overwrite = TRUE, verbose = FALSE)

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900



In [12]:
runBASiCS_dropseq <- function(seurat, stat){
    Idents(seurat) <- 'group'
    dat <- seurat
    dat$batch <- 1
    dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)),
    colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 1000, Thin = 10, Burn = 500, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = FALSE, Regression = TRUE,Threads = getOption("Ncpus", 10),)
    df = as.data.frame(displayChainBASiCS(Chain, Param = stat))
    return(df)
}

In [ ]:
data_obj <- LoadH5Seurat('dropseq.h5seurat', verbose = FALSE)

mu <- runBASiCS_dropseq(data_obj, 'mu')
write_csv(mu, file = 'dropseq_BASiCS_mu.csv')

delta <- runBASiCS_dropseq(data_obj, 'delta')
write_csv(delta, file = 'dropseq_BASiCS_delta.csv')

Validating h5Seurat file

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message in .ChecksBASiCS_Data(Data, WithSpikes):
“If spike-in genes are not available, BASiCS recommends that the data contain at least 2 batches of cells (for the same population)
”
Running no spikes BASiCS sampler (regression case) ... 


